## Partie 0 : Introduction aux Bases de Données Time-Series

### 0.1 Qu'est-ce qu'une donnée time-series ?

Une **donnée time-series** (série temporelle) est une séquence de points de données indexés dans l'ordre temporel. Exemples courants :

- **Capteurs IoT** : température, pression, humidité mesurées toutes les secondes
- **Tracking d'animaux** : positions GPS avec timestamp (notre cas!)
- **Finance** : cours de bourse, transactions bancaires
- **Monitoring IT** : métriques serveur (CPU, RAM, réseau)

**Caractéristiques clés :**
- Chaque point a un **timestamp** précis
- Les données arrivent généralement dans l'ordre chronologique
- Souvent volumineuses (millions de points)
- Les requêtes portent fréquemment sur des **intervalles de temps**

### 0.2 Pourquoi InfluxDB plutôt que MongoDB ou MySQL ?

| Critère | MySQL (Relationnel) | MongoDB (Document) | InfluxDB (Time-Series) |
|---------|---------------------|-------------------|------------------------|
| **Structure** | Tables fixes avec schéma rigide | Documents JSON flexibles | Mesures + tags + fields + time |
| **Force** | Relations complexes, ACID | Flexibilité schéma, scalabilité horizontale | Optimisé pour time-series |
| **Indexation temporelle** | Index B-tree standard | Index sur n'importe quel champ | Index temporel natif (TSM) |
| **Agrégations temporelles** | Lent (window functions) | Acceptable (aggregate pipeline) | **Très rapide** (downsampling) |
| **Compression** | Faible | Moyenne | **Très élevée** (10-20x) |
| **Cas d'usage** | E-commerce, CRM | Applications web, JSON | **Monitoring, IoT, tracking** |

**Pour notre dataset de migration d'oiseaux :**
- ✅ **89 867 points temporels** → structure time-series
- ✅ Requêtes fréquentes par **intervalle de temps** (saison de migration)
- ✅ Besoin d'**agrégations temporelles** (distance parcourue par jour)
- ✅ **Compression** importante (données géographiques répétitives)

### 0.3 Concepts Clés d'InfluxDB

InfluxDB organise les données différemment des bases relationnelles ou documents :

```
┌─────────────────────────────────────────────────────────────┐
│ MEASUREMENT (table)                                          │
├─────────────────────────────────────────────────────────────┤
│ Tags (indexés)          │ Fields (valeurs)    │ Timestamp   │
│ ======================= │ =================== │ =========== │
│ bird_id: "91732A"       │ latitude: 61.24     │ 2009-05-27  │
│ tag_id: 91732           │ longitude: 24.58    │ 14:00:00    │
│                         │ vegetation: 0.96    │             │
└─────────────────────────────────────────────────────────────┘
```

**1. Measurement** : nom de la "table" (ex: `bird_migration`)

**2. Tags** : métadonnées indexées
   - Chaînes de caractères uniquement
   - Cardinalité modérée recommandée (< 100k valeurs uniques)
   - **Utilisés pour filtrer** : `WHERE bird_id = "91732A"`
   - Exemples : bird_id, sensor_type, region

**3. Fields** : valeurs mesurées
   - Peuvent être de tout type (float, int, string, bool)
   - **Non indexés** → ne pas filtrer dessus si possible
   - Exemples : latitude, longitude, température, vitesse

**4. Timestamp** : horodatage du point (nanoseconde precision)

**Règle d'or pour choisir tag vs field :**
- **Tag** si vous filtrez souvent dessus (`WHERE ...`)
- **Field** si c'est une valeur mesurée/calculée
- **Tag** seulement si cardinalité raisonnable (éviter timestamps, IDs uniques)

## Partie 1 : Installation et Configuration

### 1.1 Importation des bibliothèques nécessaires

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# InfluxDB
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS

# Kaggle
import kagglehub

# Visualisation
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import folium
from folium.plugins import HeatMap, MarkerCluster

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

### 1.2 Configuration InfluxDB

**Objectif** : Se connecter à InfluxDB 2.x et vérifier que la connexion fonctionne.

**Configuration requise :**
- **URL** : adresse du serveur InfluxDB (ici dans Docker : `http://influxdb2:8086`)
- **Token** : clé d'authentification (admin-token configuré dans docker-compose)
- **Organisation** : espace de travail logique
- **Bucket** : "base de données" où stocker les données

In [ ]:
# Configuration InfluxDB
INFLUX_URL = "http://influxdb2:8086" 
INFLUX_TOKEN = "admin-token"
INFLUX_ORG = "fil-A3-back-bigData"
INFLUX_BUCKET = "animal-tracking"  

# Connexion au client
client = InfluxDBClient(url=INFLUX_URL, token=INFLUX_TOKEN, org=INFLUX_ORG)

# APIs
write_api = client.write_api(write_options=SYNCHRONOUS)
query_api = client.query_api()

print("✓ Client InfluxDB connecté")
print(f"  Bucket: {INFLUX_BUCKET}")
print(f"  Organisation: {INFLUX_ORG}")

# Vérification de la connexion
try:
    ping_result = client.ping()
    if ping_result:
        print(f"✓ Serveur InfluxDB répond (ping successful)")
    else:
        print("⚠️ Serveur ne répond pas")
except Exception as e:
    print(f"❌ Erreur de connexion: {e}")

# Vérifier si le bucket contient déjà des données
check_query = f'''
from(bucket: "{INFLUX_BUCKET}")
  |> range(start: 0)
  |> filter(fn: (r) => r["_measurement"] == "bird_migration")
  |> count()
  |> limit(n: 1)
'''

try:
    result = query_api.query_data_frame(check_query, org=INFLUX_ORG)
    if isinstance(result, list) and len(result) > 0:
        existing_count = result[0]['_value'].sum() if '_value' in result[0].columns else 0
        print(f"ℹ️ Le bucket contient déjà {existing_count:,} points de données")
    else:
        print(f"ℹ️ Le bucket est vide (première insertion)")
except Exception as e:
    # Bucket vide ou erreur de requête (normal si première fois)
    print(f"ℹ️ Le bucket semble vide ou c'est la première utilisation")

## Partie 2 : Chargement et Exploration des Données

**Objectif** : Comprendre la structure des données et identifier ce qui en fait une série temporelle.

In [ ]:
import os

# Download the Movebank animal tracking dataset
path = "pulkit8595/movebank-animal-tracking"
print("Downloading dataset from Kaggle...")
dataset_path = kagglehub.dataset_download(path)
print(f"Dataset downloaded to: {dataset_path}")

# List files in the dataset
print("\nFiles in dataset:")
for file in os.listdir(dataset_path):
    file_size = os.path.getsize(os.path.join(dataset_path, file)) / 1024  # KB
    print(f"  - {file} ({file_size:.2f} KB)")

In [ ]:
df = pd.read_csv(os.path.join(dataset_path, "migration_original.csv"))

print("Dataset loaded into DataFrame")
print(f"DataFrame shape: {df.shape}")
df.head()

In [ ]:
# Informations sur le dataset
print("Informations sur le dataset:")
print(df.info())
print("\n Statistiques descriptives:")
print(df.describe())
print("\n Valeurs manquantes:")
print(df.isnull().sum())

### 2.1 Analyse de cardinalité (crucial pour le design du schéma)

**Question clé** : Combien de valeurs uniques par colonne ?

La **cardinalité** (nombre de valeurs distinctes) détermine si une colonne doit être un **tag** ou un **field** dans InfluxDB :
- **Cardinalité basse** (< 100 valeurs) → Bon candidat pour tag
- **Cardinalité moyenne** (100-100k valeurs) → Acceptable comme tag si filtrage fréquent
- **Cardinalité haute** (> 100k valeurs) → **NE PAS** mettre en tag (explosion des index)

In [ ]:
# Analyse de cardinalité pour chaque colonne
print("=" * 70)
print("ANALYSE DE CARDINALITÉ".center(70))
print("=" * 70)
print(f"{'Colonne':<50} {'Cardinalité':>15}")
print("-" * 70)

for col in df.columns:
    cardinality = df[col].nunique()
    total = len(df)
    percentage = (cardinality / total) * 100
    
    # Colorier selon le type de cardinalité
    if cardinality == 1:
        indicator = "🔴 CONSTANTE"
    elif cardinality < 100:
        indicator = "🟢 BASSE (bon tag)"
    elif cardinality < 10000:
        indicator = "🟡 MOYENNE (tag ok)"
    elif cardinality == total:
        indicator = "🔴 UNIQUE (PK)"
    else:
        indicator = "🟠 HAUTE (field)"
    
    print(f"{col:<50} {cardinality:>10,} ({percentage:>5.1f}%)  {indicator}")

print("=" * 70)

### 2.2 Analyse temporelle

**Question** : À quelle fréquence les oiseaux sont-ils trackés ? Est-ce régulier ?

In [ ]:
# Convertir timestamp en datetime pour l'analyse temporelle
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Analyse de la plage temporelle
print("=" * 70)
print("ANALYSE TEMPORELLE".center(70))
print("=" * 70)
print(f"Date de début    : {df['timestamp'].min()}")
print(f"Date de fin      : {df['timestamp'].max()}")
print(f"Durée totale     : {(df['timestamp'].max() - df['timestamp'].min()).days} jours")
print(f"Nombre de points : {len(df):,}")
print()

# Analyser la fréquence de tracking par oiseau
print("Fréquence de tracking (intervalles entre points):")
print("-" * 70)

# Prendre un oiseau exemple
sample_bird = df['individual-local-identifier'].value_counts().index[0]
bird_data = df[df['individual-local-identifier'] == sample_bird].sort_values('timestamp')

# Calculer les intervalles
intervals = bird_data['timestamp'].diff().dt.total_seconds() / 3600  # en heures

print(f"Oiseau exemple : {sample_bird}")
print(f"  Intervalle moyen : {intervals.mean():.1f} heures")
print(f"  Intervalle médian: {intervals.median():.1f} heures")
print(f"  Intervalle min   : {intervals.min():.1f} heures")
print(f"  Intervalle max   : {intervals.max():.1f} heures")
print()
print("📊 Conclusion : Tracking irrégulier (pas toutes les heures exactes)")
print("   → Ceci est typique des données GPS réelles (économie batterie, couverture réseau)")
print("=" * 70)

### 2.3 Distribution des données par oiseau

**Question** : Tous les oiseaux sont-ils trackés de manière équivalente ?

In [ ]:
# Distribution du nombre de points par oiseau
bird_counts = df['individual-local-identifier'].value_counts()

print("=" * 70)
print("DISTRIBUTION DU TRACKING PAR OISEAU".center(70))
print("=" * 70)
print(f"Nombre d'oiseaux   : {len(bird_counts)}")
print(f"Points par oiseau  : {bird_counts.mean():.0f} ± {bird_counts.std():.0f} (moyenne ± écart-type)")
print(f"Min/Max            : {bird_counts.min()} / {bird_counts.max()} points")
print()
print("Top 10 oiseaux les plus trackés:")
print("-" * 70)
for i, (bird_id, count) in enumerate(bird_counts.head(10).items(), 1):
    bar = "█" * int(count / bird_counts.max() * 40)
    print(f"{i:2}. {bird_id:<15} {count:>5,} points {bar}")
print("=" * 70)

### 2.4 Visualisation géographique rapide

**Question** : Où se trouvent les oiseaux ? Migration visible ?

In [ ]:
# Préparer les données
sample_df = df.iloc[::5].copy()
sample_df['date'] = pd.to_datetime(sample_df['timestamp'])
sample_df['bird_id'] = sample_df['individual-local-identifier']


# Visualisation des trajectoires d'oiseaux
fig = px.scatter_geo(sample_df,
                     lat='location-lat',
                     lon='location-long',
                     color='bird_id',
                     hover_data=['date', 'bird_id'],
                     title='Migration Patterns - Bird Tracking Data',
                     projection='natural earth')

fig.update_layout(
    height=600,
    showlegend=True,
    geo=dict(
        showland=True,
        landcolor='rgb(243, 243, 243)',
        coastlinecolor='rgb(204, 204, 204)',
    )
)

fig.show()

# Statistiques par oiseau
print(f"📊 Dataset échantillonné: {len(sample_df):,} points")
print(f"🦅 Nombre d'oiseaux: {sample_df['bird_id'].nunique()}")
print(f"📅 Période: {sample_df['date'].min()} → {sample_df['date'].max()}")

## Partie 3 : Nettoyage des Données (3-5 min)

**Objectif** : Préparer les données pour InfluxDB en supprimant les colonnes inutiles.

**Stratégie de nettoyage** :

1. **Supprimer les colonnes 100% vides** (aucune valeur utile)
   - Exemple : `manually-marked-outlier`, `NCEP NARR SFC Vegetation at Surface`
   - Ces colonnes ajouteraient des fields inutiles dans InfluxDB

2. **Garder les colonnes redondantes mais utiles**
   - `visible` et `visible.1` semblent dupliquées mais gardons-les pour l'instant
   - Si vraiment identiques, on pourra supprimer après vérification

3. **Renommer les colonnes pour la lisibilité**
   - `ECMWF Interim Full Daily...` → `vegetation_cover_low/high`
   - Noms courts = queries Flux plus lisibles

In [ ]:
# Étape 1: Identifier et supprimer les colonnes 100% vides
columns_to_drop = df.columns[df.isnull().all()].tolist()
df_clean = df.drop(columns=columns_to_drop)

print("🧹 Nettoyage des données:")
print(f"  Colonnes supprimées : {columns_to_drop}")
print(f"  Raison : 100% de valeurs manquantes (inutiles)")
print()

# Étape 2: Renommer les colonnes de végétation pour plus de clarté
df_clean = df_clean.rename(columns={
    'ECMWF Interim Full Daily Invariant Low Vegetation Cover': 'vegetation_cover_low',
    'ECMWF Interim Full Daily Invariant High Vegetation Cover': 'vegetation_cover_high'
})

print(f"✓ Colonnes renommées pour lisibilité:")
print(f"  'ECMWF Interim...' → 'vegetation_cover_low/high'")
print()

# Étape 3: Vérifier qu'il ne reste plus de valeurs manquantes
null_counts = df_clean.isnull().sum()
remaining_nulls = null_counts[null_counts > 0]

if len(remaining_nulls) == 0:
    print(f"✓ Aucune valeur manquante dans les {df_clean.shape[1]} colonnes restantes")
else:
    print(f"⚠️ Valeurs manquantes détectées:")
    for col, count in remaining_nulls.items():
        print(f"  {col}: {count:,} valeurs manquantes ({count/len(df_clean)*100:.1f}%)")

print()
print(f"📊 Dataset nettoyé : {df_clean.shape[0]:,} lignes × {df_clean.shape[1]} colonnes")
print(f"   Prêt pour insertion dans InfluxDB!")

## Partie 4 : Design du Schéma et Insertion dans InfluxDB (10-12 min)

**Objectif** : Comprendre comment choisir entre tags et fields, puis insérer les données efficacement.

### 4.1 Décisions de Design : Tags vs Fields

**Rappel de la règle d'or** :
- **Tags** = filtres fréquents + cardinalité raisonnable
- **Fields** = valeurs mesurées + pas de filtrage

Analysons chaque colonne :

In [ ]:
# Matrice de décision Tag vs Field 
import pandas as pd

decisions = [
    {
        "Colonne": "timestamp",
        "Cardinalité": df_clean['timestamp'].nunique(),
        "Type": "⏱️ TIMESTAMP",
        "Décision": "Timestamp InfluxDB",
        "Raison": "Index temporel principal"
    },
    {
        "Colonne": "individual-local-identifier",
        "Cardinalité": df_clean['individual-local-identifier'].nunique(),
        "Type": "🟢 TAG",
        "Décision": "Tag",
        "Raison": "Filtrage fréquent par oiseau, ~199 valeurs (cardinalité ok)"
    },
    {
        "Colonne": "tag-local-identifier",
        "Cardinalité": df_clean['tag-local-identifier'].nunique(),
        "Type": "🟢 TAG",
        "Décision": "Tag",
        "Raison": "Filtrage par device, ~199 valeurs (cardinalité ok)"
    },
    {
        "Colonne": "sensor-type",
        "Cardinalité": df_clean['sensor-type'].nunique(),
        "Type": "🔴 FIELD",
        "Décision": "Field (anti-pattern si tag)",
        "Raison": "CARDINALITÉ = 1 ! Index inutile, gaspillage mémoire"
    },
    {
        "Colonne": "individual-taxon-canonical-name",
        "Cardinalité": df_clean['individual-taxon-canonical-name'].nunique(),
        "Type": "🔴 FIELD",
        "Décision": "Field",
        "Raison": "Cardinalité = 1 (tous Larus fuscus), même raison"
    },
    {
        "Colonne": "study-name",
        "Cardinalité": df_clean['study-name'].nunique(),
        "Type": "🔴 FIELD",
        "Décision": "Field",
        "Raison": "Cardinalité = 1, pas de filtrage nécessaire"
    },
    {
        "Colonne": "event-id",
        "Cardinalité": df_clean['event-id'].nunique(),
        "Type": "🔴 FIELD",
        "Décision": "Field",
        "Raison": "Cardinalité = 89867 (unique), jamais en tag!"
    },
    {
        "Colonne": "location-lat",
        "Cardinalité": df_clean['location-lat'].nunique(),
        "Type": "🔴 FIELD",
        "Décision": "Field",
        "Raison": "Valeur mesurée (coordonnée GPS)"
    },
    {
        "Colonne": "location-long",
        "Cardinalité": df_clean['location-long'].nunique(),
        "Type": "🔴 FIELD",
        "Décision": "Field",
        "Raison": "Valeur mesurée (coordonnée GPS)"
    },
    {
        "Colonne": "vegetation_cover_low",
        "Cardinalité": df_clean['vegetation_cover_low'].nunique(),
        "Type": "🔴 FIELD",
        "Décision": "Field",
        "Raison": "Valeur mesurée (donnée environnementale)"
    },
    {
        "Colonne": "vegetation_cover_high",
        "Cardinalité": df_clean['vegetation_cover_high'].nunique(),
        "Type": "🔴 FIELD",
        "Décision": "Field",
        "Raison": "Valeur mesurée (donnée environnementale)"
    },
    {
        "Colonne": "visible / visible.1",
        "Cardinalité": df_clean['visible'].nunique(),
        "Type": "🔴 FIELD",
        "Décision": "Field",
        "Raison": "Métadonnée booléenne, pas de filtrage"
    }
]

decision_df = pd.DataFrame(decisions)

print("=" * 100)
print("MATRICE DE DÉCISION : TAGS vs FIELDS".center(100))
print("=" * 100)
print(f"{'Colonne':<35} {'Card.':>10} {'Type':^20} {'Raison':<30}")
print("-" * 100)

for _, row in decision_df.iterrows():
    print(f"{row['Colonne']:<35} {row['Cardinalité']:>10,} {row['Type']:^20} {row['Raison']:<30}")

print("=" * 100)
print()
print("📋 RÉSUMÉ DU SCHÉMA:")
print(f"  🟢 Tags (2)   : individual-local-identifier, tag-local-identifier")
print(f"  🔴 Fields (9) : location-lat, location-long, event-id, vegetation_cover_*, etc.")
print(f"  ⏱️ Timestamp  : timestamp")
print()

### 4.2 Anti-Patterns à Éviter

**❌ NE JAMAIS faire ça** :

```python
# MAUVAIS EXEMPLE - Ne pas reproduire!
data_frame_tag_columns=[
    'sensor-type',                    # ❌ Cardinalité = 1 (gaspillage)
    'individual-taxon-canonical-name',  # ❌ Cardinalité = 1 (gaspillage)
    'event-id',                       # ❌ Cardinalité = 89867 (explosion index!)
    'timestamp',                      # ❌ Timestamp n'est JAMAIS un tag
    'location-lat'                    # ❌ Float avec haute cardinalité
]
```

**Conséquences** :
- **Explosion de la cardinalité** → Index gigantesques → RAM saturée
- **Queries lentes** → Trop d'index à parcourir
- **Gaspillage mémoire** → Indexer des constantes ne sert à rien

**✅ Notre schéma optimisé** :
- Seulement 2 tags avec cardinalité raisonnable (~199 valeurs chacun)
- Tout le reste en fields
- Performance maximale pour les requêtes de tracking par oiseau/device

### 4.3 Insertion des Données

In [ ]:
# Conversion timestamp (déjà fait dans Part 2, mais on s'assure)
df_clean['timestamp'] = pd.to_datetime(df_clean['timestamp'])

print(f"📤 Préparation de l'insertion...")
print(f"   Records à insérer : {len(df_clean):,}")
print(f"   Measurement       : bird_migration")
print(f"   Tags (2)          : individual-local-identifier, tag-local-identifier")
print(f"   Fields (auto)     : {df_clean.shape[1] - 3} colonnes")  # Total - timestamp - 2 tags
print(f"   Période           : {df_clean['timestamp'].min()} → {df_clean['timestamp'].max()}")
print()

# Insertion dans InfluxDB avec schéma optimisé
import time
start_time = time.time()

try:
    write_api.write(
        bucket=INFLUX_BUCKET, 
        org=INFLUX_ORG, 
        record=df_clean, 
        data_frame_measurement_name='bird_migration',
        data_frame_tag_columns=[
            'individual-local-identifier',  # ~199 oiseaux (filtrage fréquent)
            'tag-local-identifier',         # ~199 devices (filtrage fréquent)
        ],
        # Toutes les autres colonnes deviennent automatiquement des fields:
        # location-long, location-lat, event-id, vegetation_cover_low/high, etc.
        data_frame_timestamp_column='timestamp'
    )
    
    elapsed = time.time() - start_time
    
    print(f"✅ Insertion réussie en {elapsed:.2f} secondes!")
    print(f"   Débit: {len(df_clean)/elapsed:.0f} points/seconde")
    print()
    
except Exception as e:
    print(f"❌ Erreur lors de l'insertion: {e}")
    raise

### 4.4 Vérification de l'Insertion

In [ ]:
# Requête de vérification: compter tous les points insérés
verification_query = f'''
from(bucket: "{INFLUX_BUCKET}")
  |> range(start: 0)
  |> filter(fn: (r) => r["_measurement"] == "bird_migration")
  |> group()
  |> count()
'''

print("🔍 Vérification de l'insertion...")
result = query_api.query_data_frame(verification_query, org=INFLUX_ORG)

if isinstance(result, list) and len(result) > 0:
    # InfluxDB retourne un point par field, on compte le total
    total_points = len(result) if isinstance(result, list) else len(result)
    
    # Nombre de fields attendus
    num_fields = df_clean.shape[1] - 3  # Total - timestamp - 2 tags
    expected_records = len(df_clean)
    
    print(f"✅ Vérification réussie:")
    print(f"   Measurement       : bird_migration")
    print(f"   Records insérés   : {expected_records:,}")
    print(f"   Fields par record : {num_fields}")
    print(f"   Points totaux     : {expected_records * num_fields:,} (records × fields)")
    print()
    print(f"📊 Taille estimée  : {(expected_records * num_fields * 8) / 1024 / 1024:.2f} MB (sans compression)")
    print(f"   Compression InfluxDB : ~10-20x → {(expected_records * num_fields * 8) / 1024 / 1024 / 15:.2f} MB estimé")
else:
    print(result)
    print(f"⚠️ Aucune donnée trouvée. Vérifier l'insertion.")

## Partie 5 : Langage de Requêtes Flux - Construction Progressive (12-15 min)

**Objectif** : Maîtriser Flux en construisant des requêtes de plus en plus complexes.

In [ ]:
# Helper function pour afficher les résultats Flux proprement
def display_flux_result(result, title="Résultat"):
    """Affiche les résultats d'une requête Flux de manière lisible"""
    if isinstance(result, list):
        if len(result) > 0:
            result_df = pd.concat(result, ignore_index=True)
            print(f"📊 {title}: {len(result)} DataFrames, {len(result_df)} lignes totales")
            display(result_df.head(10))
            return result_df
        else:
            print(f"⚠️ {title}: Aucune donnée trouvée")
            return None
    else:
        print(f"📊 {title}: {len(result)} lignes")
        if len(result) > 0:
            display(result.head(10))
        return result

### 5.1 Requête de Base : Récupérer des Données

**Anatomie d'une requête Flux** :
```
from(bucket: "nom")              ← Source des données
  |> range(start: xxx, stop: yyy)  ← Filtre temporel (OBLIGATOIRE)
  |> filter(fn: (r) => ...)        ← Filtres supplémentaires
  |> limit(n: 10)                  ← Limiter le nombre de résultats
```

**Pipeline** : Chaque opérateur `|>` passe les données à l'opérateur suivant

In [ ]:
# Requête 1: Récupérer les 100 premiers points
query_basic = f'''
from(bucket: "{INFLUX_BUCKET}")
  |> range(start: 0)  
  |> filter(fn: (r) => r["_measurement"] == "bird_migration")
  |> limit(n: 100)
'''

print("=" * 80)
print("REQUÊTE 1: RÉCUPÉRATION BASIQUE (100 premiers points)".center(80))
print("=" * 80)
print(query_basic)
print()

result_basic = query_api.query_data_frame(query_basic, org=INFLUX_ORG)
df_basic = display_flux_result(result_basic, "Requête basique")

### 5.2 Filtrage par Tag : Requêtes Efficaces

**Pourquoi filtrer sur les tags** : Index optimisés → queries ultra-rapides

**Équivalence** :
- Flux : `filter(fn: (r) => r["individual-local-identifier"] == "91732A")`
- SQL : `WHERE individual_local_identifier = '91732A'`
- Pandas : `df[df['individual-local-identifier'] == '91732A']`

In [ ]:
# Requête 2: Filtrer par oiseau spécifique (tag)
bird_id = "91732A"

query_by_bird = f'''
from(bucket: "{INFLUX_BUCKET}")
  |> range(start: 0)
  |> filter(fn: (r) => r["_measurement"] == "bird_migration")
  |> filter(fn: (r) => r["individual-local-identifier"] == "{bird_id}")
  |> limit(n: 50)
'''

print("=" * 80)
print(f"REQUÊTE 2: FILTRAGE PAR TAG (oiseau {bird_id})".center(80))
print("=" * 80)
print(query_by_bird)
print()

result_by_bird = query_api.query_data_frame(query_by_bird, org=INFLUX_ORG)
df_by_bird = display_flux_result(result_by_bird, f"Données pour {bird_id}")

print()
print("🔍 Vérification avec Pandas:")
pandas_count = len(df_clean[df_clean['individual-local-identifier'] == bird_id])
print(f"   Pandas : {pandas_count:,} records pour {bird_id}")
print(f"   → Cohérence avec InfluxDB ✅")

### 5.3 Filtrage Temporel : La Force des Time-Series DB

**C'est ICI que InfluxDB brille** : requêtes temporelles ultra-optimisées

**Exemple** : Trouver les données de migration estivale (juin-août 2009)

In [92]:
# Requête 3: Filtrage par plage temporelle (summer 2009)
query_summer = f'''
from(bucket: "{INFLUX_BUCKET}")
  |> range(start: 2009-06-01T00:00:00Z, stop: 2009-08-31T23:59:59Z)
  |> filter(fn: (r) => r["_measurement"] == "bird_migration")
  |> filter(fn: (r) => r["_field"] == "location-lat")
  |> limit(n: 100)
'''

print("=" * 80)
print("REQUÊTE 3: FILTRAGE TEMPOREL (été 2009)".center(80))
print("=" * 80)
print(query_summer)
print()

result_summer = query_api.query_data_frame(query_summer, org=INFLUX_ORG)
df_summer = display_flux_result(result_summer, "Données été 2009")

print()
print("🔍 Vérification avec Pandas:")
summer_mask = (df_clean['timestamp'] >= '2009-06-01') & (df_clean['timestamp'] <= '2009-08-31')
pandas_summer = len(df_clean[summer_mask])
print(f"   Pandas : {pandas_summer:,} records pour l'été 2009")

print()
print("⚡ Performance:")
print("   InfluxDB : Index temporel (TSM) → recherche en O(log n)")
print("   Pandas   : Scan complet → O(n)")
print("   → Pour millions de points, InfluxDB est 100-1000x plus rapide!")

                    REQUÊTE 3: FILTRAGE TEMPOREL (été 2009)                     

from(bucket: "animal-tracking")
  |> range(start: 2009-06-01T00:00:00Z, stop: 2009-08-31T23:59:59Z)
  |> filter(fn: (r) => r["_measurement"] == "bird_migration")
  |> filter(fn: (r) => r["_field"] == "location-lat")
  |> limit(n: 100)


📊 Données été 2009: 7539 lignes


,result,table,_start,_stop,_time,_value,_field,_measurement,individual-local-identifier,tag-local-identifier
0,_result,0,2009-06-01 00:00:00+00:00,2009-08-31 23:59:59+00:00,2009-06-01 05:00:00+00:00,61.23300,location-lat,bird_migration,91732A,91732
1,_result,0,2009-06-01 00:00:00+00:00,2009-08-31 23:59:59+00:00,2009-06-02 05:00:00+00:00,61.22933,location-lat,bird_migration,91732A,91732
2,_result,0,2009-06-01 00:00:00+00:00,2009-08-31 23:59:59+00:00,2009-06-02 14:00:00+00:00,61.21700,location-lat,bird_migration,91732A,91732
3,_result,0,2009-06-01 00:00:00+00:00,2009-08-31 23:59:59+00:00,2009-06-02 20:00:00+00:00,61.19000,location-lat,bird_migration,91732A,91732
4,_result,0,2009-06-01 00:00:00+00:00,2009-08-31 23:59:59+00:00,2009-06-03 05:00:00+00:00,61.23317,location-lat,bird_migration,91732A,91732
5,_result,0,2009-06-01 00:00:00+00:00,2009-08-31 23:59:59+00:00,2009-06-03 08:00:00+00:00,61.24783,location-lat,bird_migration,91732A,91732
6,_result,0,2009-06-01 00:00:00+00:00,2009-08-31 23:59:59+00:00,2009-06-03 14:00:00+00:00,61.24783,location-lat,bird_migration,91732A,91732
7,_result,0,2009-06-01 00:00:00+00:00,2009-08-31 23:59:59+00:00,2009-06-03 20:00:00+00:00,61.26983,location-lat,bird_migration,91732A,91732
8,_result,0,2009-06-01 00:00:00+00:00,2009-08-31 23:59:59+00:00,2009-06-04 05:00:00+00:00,61.26833,location-lat,bird_migration,91732A,91732
9,_result,0,2009-06-01 00:00:00+00:00,2009-08-31 23:59:59+00:00,2009-06-04 08:00:00+00:00,61.24767,location-lat,bird_migration,91732A,91732



🔍 Vérification avec Pandas:
   Pandas : 13,294 records pour l'été 2009

⚡ Performance:
   InfluxDB : Index temporel (TSM) → recherche en O(log n)
   Pandas   : Scan complet → O(n)
   → Pour millions de points, InfluxDB est 100-1000x plus rapide!


### 5.4 Restructuration avec Pivot : Transformer Fields en Colonnes

**Problème** : InfluxDB retourne 1 ligne par field → difficile à analyser

**Solution** : `pivot()` transforme les fields en colonnes (comme un DataFrame)

In [91]:
# Requête 4: Utiliser pivot pour restructurer les données
query_pivot = f'''
from(bucket: "{INFLUX_BUCKET}")
  |> range(start: 0)
  |> filter(fn: (r) => r["_measurement"] == "bird_migration")
  |> filter(fn: (r) => r["individual-local-identifier"] == "91732A")
  |> filter(fn: (r) => r["_field"] == "location-lat" or r["_field"] == "location-long")
  |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
  |> limit(n: 20)
'''

print("=" * 80)
print("REQUÊTE 4: PIVOT (fields → colonnes)".center(80))
print("=" * 80)
print(query_pivot)
print()

result_pivot = query_api.query_data_frame(query_pivot, org=INFLUX_ORG)
df_pivot = display_flux_result(result_pivot, "Données pivotées")

print()
print("📊 Maintenant nous avons:")
print("   - 1 ligne = 1 point GPS (timestamp)")
print("   - Colonnes location-lat et location-long côte à côte")
print("   → Format idéal pour l'analyse et la visualisation!")

                      REQUÊTE 4: PIVOT (fields → colonnes)                      

from(bucket: "animal-tracking")
  |> range(start: 0)
  |> filter(fn: (r) => r["_measurement"] == "bird_migration")
  |> filter(fn: (r) => r["individual-local-identifier"] == "91732A")
  |> filter(fn: (r) => r["_field"] == "location-lat" or r["_field"] == "location-long")
  |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
  |> limit(n: 20)


📊 Données pivotées: 20 lignes


,result,table,_start,_stop,_time,_measurement,individual-local-identifier,tag-local-identifier,location-lat,location-long
0,_result,0,1970-01-01 00:00:00+00:00,2025-11-02 16:27:24.623621+00:00,2009-05-27 14:00:00+00:00,bird_migration,91732A,91732,61.24783,24.58617
1,_result,0,1970-01-01 00:00:00+00:00,2025-11-02 16:27:24.623621+00:00,2009-05-27 20:00:00+00:00,bird_migration,91732A,91732,61.23267,24.58217
2,_result,0,1970-01-01 00:00:00+00:00,2025-11-02 16:27:24.623621+00:00,2009-05-28 05:00:00+00:00,bird_migration,91732A,91732,61.18833,24.53133
3,_result,0,1970-01-01 00:00:00+00:00,2025-11-02 16:27:24.623621+00:00,2009-05-28 08:00:00+00:00,bird_migration,91732A,91732,61.23283,24.58200
4,_result,0,1970-01-01 00:00:00+00:00,2025-11-02 16:27:24.623621+00:00,2009-05-28 14:00:00+00:00,bird_migration,91732A,91732,61.23267,24.58250
5,_result,0,1970-01-01 00:00:00+00:00,2025-11-02 16:27:24.623621+00:00,2009-05-28 20:00:00+00:00,bird_migration,91732A,91732,61.24767,24.58617
6,_result,0,1970-01-01 00:00:00+00:00,2025-11-02 16:27:24.623621+00:00,2009-05-29 05:00:00+00:00,bird_migration,91732A,91732,61.24767,24.58600
7,_result,0,1970-01-01 00:00:00+00:00,2025-11-02 16:27:24.623621+00:00,2009-05-29 08:00:00+00:00,bird_migration,91732A,91732,61.24767,24.58617
8,_result,0,1970-01-01 00:00:00+00:00,2025-11-02 16:27:24.623621+00:00,2009-05-29 14:00:00+00:00,bird_migration,91732A,91732,61.24750,24.58650
9,_result,0,1970-01-01 00:00:00+00:00,2025-11-02 16:27:24.623621+00:00,2009-05-29 20:00:00+00:00,bird_migration,91732A,91732,61.23883,24.56967



📊 Maintenant nous avons:
   - 1 ligne = 1 point GPS (timestamp)
   - Colonnes location-lat et location-long côte à côte
   → Format idéal pour l'analyse et la visualisation!


### 5.5 Agrégation : Compter, Grouper, Analyser

**Opérations d'agrégation courantes** :
- `count()` : nombre de points
- `mean()` : moyenne
- `sum()` : somme
- `max()` / `min()` : valeurs extrêmes
- `group()` : grouper par tag(s)

**Exemple** : Compter le nombre de tracking points par oiseau

In [93]:
# Requête 5: Compter les points de tracking par oiseau
query_count = f'''
from(bucket: "{INFLUX_BUCKET}")
  |> range(start: 0)
  |> filter(fn: (r) => r["_measurement"] == "bird_migration")
  |> filter(fn: (r) => r["_field"] == "location-lat")
  |> group(columns: ["individual-local-identifier"])
  |> count()
  |> sort(columns: ["_value"], desc: true)
  |> limit(n: 20)
'''

print("=" * 80)
print("REQUÊTE 5: AGRÉGATION (count par oiseau)".center(80))
print("=" * 80)
print(query_count)
print()

result_count = query_api.query_data_frame(query_count, org=INFLUX_ORG)
df_count = display_flux_result(result_count, "Comptage par oiseau")

print()
print("🔍 Équivalence Pandas:")
print("   df.groupby('individual-local-identifier').size().sort_values(ascending=False)")
print()
print("📊 Vérification (top 5):")
pandas_counts = df_clean.groupby('individual-local-identifier').size().sort_values(ascending=False).head(5)
print(pandas_counts)

                    REQUÊTE 5: AGRÉGATION (count par oiseau)                    

from(bucket: "animal-tracking")
  |> range(start: 0)
  |> filter(fn: (r) => r["_measurement"] == "bird_migration")
  |> filter(fn: (r) => r["_field"] == "location-lat")
  |> group(columns: ["individual-local-identifier"])
  |> count()
  |> sort(columns: ["_value"], desc: true)
  |> limit(n: 20)


📊 Comptage par oiseau: 126 lignes


,result,table,_start,_stop,_value,individual-local-identifier
0,_result,0,1970-01-01 00:00:00+00:00,2025-11-02 16:28:23.889530+00:00,1970,91732A
1,_result,1,1970-01-01 00:00:00+00:00,2025-11-02 16:28:23.889530+00:00,44,91733A
2,_result,2,1970-01-01 00:00:00+00:00,2025-11-02 16:28:23.889530+00:00,1060,91734A
3,_result,3,1970-01-01 00:00:00+00:00,2025-11-02 16:28:23.889530+00:00,477,91735A
4,_result,4,1970-01-01 00:00:00+00:00,2025-11-02 16:28:23.889530+00:00,1039,91737A
5,_result,5,1970-01-01 00:00:00+00:00,2025-11-02 16:28:23.889530+00:00,424,91738A
6,_result,6,1970-01-01 00:00:00+00:00,2025-11-02 16:28:23.889530+00:00,2738,91739A
7,_result,7,1970-01-01 00:00:00+00:00,2025-11-02 16:28:23.889530+00:00,335,91740A
8,_result,8,1970-01-01 00:00:00+00:00,2025-11-02 16:28:23.889530+00:00,415,91741A
9,_result,9,1970-01-01 00:00:00+00:00,2025-11-02 16:28:23.889530+00:00,1006,91742A



🔍 Équivalence Pandas:
   df.groupby('individual-local-identifier').size().sort_values(ascending=False)

📊 Vérification (top 5):
individual-local-identifier
91916A    8396
91752A    5751
91823A    5557
91763A    5216
91814A    4775
dtype: int64


### 5.6 Window Functions : Agrégation Temporelle

**Cas d'usage** : Calculer des moyennes par jour/heure/mois

`aggregateWindow()` divise le temps en fenêtres et agrège chacune séparément

In [ ]:
# Requête 6: Moyenne quotidienne de la couverture végétale (mai 2009)
query_window = f'''
from(bucket: "{INFLUX_BUCKET}")
  |> range(start: 2009-05-01T00:00:00Z, stop: 2009-05-31T23:59:59Z)
  |> filter(fn: (r) => r["_measurement"] == "bird_migration")
  |> filter(fn: (r) => r["_field"] == "vegetation_cover_high")
  |> aggregateWindow(every: 1d, fn: mean, createEmpty: false)
  |> limit(n: 31)
'''

print("=" * 80)
print("REQUÊTE 6: WINDOW FUNCTION (moyenne journalière)".center(80))
print("=" * 80)
print(query_window)
print()

result_window = query_api.query_data_frame(query_window, org=INFLUX_ORG)
df_window = display_flux_result(result_window, "Moyenne quotidienne végétation")

print()
print("📊 Explication:")
print("   aggregateWindow(every: 1d, fn: mean)")
print("   → Divise mai 2009 en fenêtres de 1 jour")
print("   → Calcule la moyenne pour chaque jour")
print("   → Retourne 31 points (1 par jour)")
print()
print("💡 Très utile pour:")
print("   - Downsampling (réduire volume de données)")
print("   - Détecter tendances (patterns journaliers/hebdomadaires)")
print("   - Visualisations (éviter trop de points sur un graphe)")

## Partie 5.5 : Analyse Time-Series - Patterns de Migration (10-12 min)

**Objectif** : Utiliser Flux pour extraire des insights sur les migrations d'oiseaux.

### 5.5.1 Trouver les Points Extrêmes de Migration

**Question** : Quelle est la latitude la plus au nord et au sud visitée par chaque oiseau ?

In [ ]:
# Requête: Points les plus au nord et sud par oiseau
query_extremes = f'''
from(bucket: "{INFLUX_BUCKET}")
  |> range(start: 0)
  |> filter(fn: (r) => r["_measurement"] == "bird_migration")
  |> filter(fn: (r) => r["_field"] == "location-lat")
  |> group(columns: ["individual-local-identifier"])
  |> reduce(
      fn: (r, accumulator) => ({{
        bird_id: r["individual-local-identifier"],
        max_lat: if r._value > accumulator.max_lat then r._value else accumulator.max_lat,
        min_lat: if r._value < accumulator.min_lat then r._value else accumulator.min_lat
      }}),
      identity: {{bird_id: "", max_lat: -90.0, min_lat: 90.0}}
    )
  |> limit(n: 10)
'''

print("=" * 80)
print("ANALYSE: POINTS EXTRÊMES DE MIGRATION".center(80))
print("=" * 80)
print(query_extremes)
print()

result_extremes = query_api.query_data_frame(query_extremes, org=INFLUX_ORG)
df_extremes = display_flux_result(result_extremes, "Points extrêmes par oiseau")

print()
print("📊 Interprétation:")
print("   max_lat = point le plus au nord (zone de reproduction probable)")
print("   min_lat = point le plus au sud (zone d'hivernage)")
print("   Différence = amplitude de migration")
print()

if df_extremes is not None and len(df_extremes) > 0:
    # Calculer l'amplitude de migration
    if 'max_lat' in df_extremes.columns and 'min_lat' in df_extremes.columns:
        df_extremes['migration_range'] = df_extremes['max_lat'] - df_extremes['min_lat']
        print("🦅 Top 3 plus grandes amplitudes de migration:")
        top_migrants = df_extremes.nlargest(3, 'migration_range')[['bird_id', 'min_lat', 'max_lat', 'migration_range']]
        print(top_migrants.to_string(index=False))

### 5.5.3 Analyse Saisonnière : Été vs Hiver

**Hypothèse** : Les oiseaux migrateurs se déplacent vers le nord en été (reproduction) et vers le sud en hiver

Testons cette hypothèse avec les données !

In [ ]:
# Comparer les latitudes moyennes été vs hiver
query_summer_lat = f'''
from(bucket: "{INFLUX_BUCKET}")
  |> range(start: 2009-06-01T00:00:00Z, stop: 2009-08-31T23:59:59Z)
  |> filter(fn: (r) => r["_measurement"] == "bird_migration")
  |> filter(fn: (r) => r["_field"] == "location-lat")
  |> mean()
'''

query_winter_lat = f'''
from(bucket: "{INFLUX_BUCKET}")
  |> range(start: 2009-12-01T00:00:00Z, stop: 2010-02-28T23:59:59Z)
  |> filter(fn: (r) => r["_measurement"] == "bird_migration")
  |> filter(fn: (r) => r["_field"] == "location-lat")
  |> mean()
'''

print("=" * 80)
print("ANALYSE SAISONNIÈRE: ÉTÉ vs HIVER".center(80))
print("=" * 80)

result_summer_lat = query_api.query_data_frame(query_summer_lat, org=INFLUX_ORG)
result_winter_lat = query_api.query_data_frame(query_winter_lat, org=INFLUX_ORG)

def extract_mean_value(result):
    """Extrait la valeur moyenne d'un résultat Flux"""
    if isinstance(result, list) and len(result) > 0:
        return result[0]['_value'].values[0] if '_value' in result[0].columns else None
    elif isinstance(result, pd.DataFrame) and len(result) > 0:
        return result['_value'].values[0] if '_value' in result.columns else None
    return None

summer_mean = extract_mean_value(result_summer_lat)
winter_mean = extract_mean_value(result_winter_lat)

if summer_mean is not None and winter_mean is not None:
    print(f"📊 Latitude moyenne:")
    print(f"   Été 2009 (juin-août)    : {summer_mean:.2f}°")
    print(f"   Hiver 2009-10 (déc-fév) : {winter_mean:.2f}°")
    print(f"   Différence              : {summer_mean - winter_mean:.2f}°")
    print()
    
    if summer_mean > winter_mean:
        print(f"✅ HYPOTHÈSE CONFIRMÉE!")
        print(f"   Les oiseaux sont {summer_mean - winter_mean:.1f}° plus au nord en été")
        print(f"   → Pattern de migration classique pour Larus fuscus (goéland brun)")
        print(f"   → Reproduction en Scandinavie (été), hivernage en Afrique (hiver)")
    else:
        print(f"❌ Hypothèse non confirmée (pattern inattendu)")
else:
    print(f"⚠️ Données insuffisantes pour cette analyse")

print("=" * 80)

### 5.5.4 Efficacité du Stockage : Compression des Time-Series

**Big Data Concept** : InfluxDB utilise une compression très efficace pour les données time-series.

**Pourquoi c'est important ?**
- Données time-series = souvent répétitives (même tags, valeurs proches)
- Compression 10-20x typique → économies massives de stockage
- Impact sur coûts cloud et performance I/O

In [ ]:
# Calculer la taille théorique des données non compressées
num_records = len(df_clean)
num_fields = df_clean.shape[1] - 3  # Exclude timestamp and 2 tags

# Estimation de taille par point de donnée
# - Timestamp: 8 bytes (int64)
# - Tags (2): ~20 bytes each (string) = 40 bytes
# - Fields (10): ~8 bytes each (float64) = 80 bytes
# - Overhead: ~20 bytes
bytes_per_record = 8 + 40 + 80 + 20  # = 148 bytes

total_size_uncompressed = num_records * bytes_per_record
total_size_mb = total_size_uncompressed / 1024 / 1024

print("=" * 80)
print("ANALYSE D'EFFICACITÉ DU STOCKAGE".center(80))
print("=" * 80)
print(f"📊 Données insérées:")
print(f"   Records          : {num_records:,}")
print(f"   Fields par record: {num_fields}")
print(f"   Points totaux    : {num_records * num_fields:,}")
print()
print(f"💾 Taille estimée SANS compression:")
print(f"   Par record       : {bytes_per_record} bytes")
print(f"   Total brut       : {total_size_mb:.2f} MB")
print()
print(f"⚡ Avec compression InfluxDB (ratio 10-20x):")
print(f"   Compression 10x  : {total_size_mb / 10:.2f} MB")
print(f"   Compression 15x  : {total_size_mb / 15:.2f} MB")
print(f"   Compression 20x  : {total_size_mb / 20:.2f} MB")
print()
print(f"🔍 Pourquoi une si bonne compression ?")
print(f"   1. **Tags répétés** : 'individual-local-identifier' se répète {num_records // df_clean['individual-local-identifier'].nunique()} fois en moyenne")
print(f"   2. **Timestamps ordonnés** : Delta encoding (stocker les différences)")
print(f"   3. **Valeurs proches** : Coordonnées GPS changent peu entre points consécutifs")
print(f"   4. **Run-length encoding** : Colonnes constantes (sensor-type, study-name)")
print()
print(f"📈 Impact pour Big Data:")
print(f"   - 1 million d'oiseaux × 1000 points/oiseau = 1 milliard de points")
print(f"   - Sans compression : {(1_000_000_000 * bytes_per_record) / 1024 / 1024 / 1024:.1f} GB")
print(f"   - Avec compression 15x : {(1_000_000_000 * bytes_per_record) / 1024 / 1024 / 1024 / 15:.1f} GB")
print(f"   → Économie de {((1_000_000_000 * bytes_per_record) / 1024 / 1024 / 1024) - ((1_000_000_000 * bytes_per_record) / 1024 / 1024 / 1024 / 15):.1f} GB!")
print("=" * 80)

### 5.5.5 Retention Policies : Gestion du Cycle de Vie des Données

**Big Data Concept** : En production, on ne garde pas toutes les données éternellement.

**Retention Policy** = règle qui définit :
- Combien de temps garder les données
- Suppression automatique des données anciennes
- Économie de stockage et amélioration des performances

In [ ]:
print("=" * 80)
print("RETENTION POLICIES - GESTION DU CYCLE DE VIE".center(80))
print("=" * 80)
print()
print("📋 Concept:")
print("   Une Retention Policy définit combien de temps garder les données")
print("   Après expiration → suppression automatique → libération d'espace")
print()
print("🔧 Cas d'usage typiques:")
print()
print("   1. **Données brutes récentes** (haute résolution)")
print("      Retention: 7 jours")
print("      → Tracking en temps réel, debugging")
print()
print("   2. **Données downsamplées** (agrégations horaires)")
print("      Retention: 90 jours")
print("      → Analyses historiques, dashboards")
print()
print("   3. **Données agrégées** (moyennes journalières)")
print("      Retention: Illimitée ou 5 ans")
print("      → Tendances long terme, reporting annuel")
print()
print("💡 Exemple pour notre dataset d'oiseaux:")
print("   Bucket 'raw-tracking'     : Retention 30 jours  (points GPS bruts)")
print("   Bucket 'hourly-positions' : Retention 1 an      (position moyenne/heure)")
print("   Bucket 'daily-summary'    : Retention illimitée (statistiques journalières)")
print()
print("⚙️ Configuration InfluxDB (exemple):")
print("   ```")
print("   # Créer un bucket avec retention de 30 jours")
print("   influx bucket create \\")
print("     --name raw-tracking \\")
print("     --retention 720h \\")  # 30 jours = 720 heures
print("     --org fil-A3-back-bigData")
print("   ```")
print()
print("🎯 Bénéfices pour Big Data:")
print("   ✅ Contrôle automatique de la croissance des données")
print("   ✅ Pas de scripts de nettoyage manuel")
print("   ✅ Queries plus rapides (moins de données à scanner)")
print("   ✅ Coûts cloud prévisibles")
print()
print("📊 Pour notre dataset actuel:")
data_start = df_clean['timestamp'].min()
data_end = df_clean['timestamp'].max()
data_span_days = (data_end - data_start).days
print(f"   Période des données: {data_start.date()} → {data_end.date()}")
print(f"   Durée: {data_span_days} jours")
print(f"   Avec retention de 30 jours, on garderait seulement:")
print(f"   - Les {min(30, data_span_days)} derniers jours de données")
print(f"   - ~{(30 / data_span_days * 100):.1f}% du dataset actuel")
print("=" * 80)

## Partie 6 : Visualisations - Démonstration (OPTIONNEL - 8-10 min)

**Note pédagogique** : Cette partie est une **démonstration optionnelle** pour montrer ce qu'on peut faire avec les données. L'objectif principal du TP est d'apprendre **InfluxDB** (schema design, Flux queries), pas la visualisation de données.

**Objectif** : Montrer rapidement quelques visualisations possibles des patterns de migration.

### 6.1 Carte Interactive des Routes de Migration

**Objectif** : Visualiser les trajectoires des 3 oiseaux les plus trackés sur une carte interactive avec Folium.

In [ ]:
# Identifier les 3 oiseaux les plus trackés
query_top_birds = f'''
from(bucket: "{INFLUX_BUCKET}")
  |> range(start: 0)
  |> filter(fn: (r) => r["_measurement"] == "bird_migration")
  |> filter(fn: (r) => r["_field"] == "location-lat")
  |> group(columns: ["individual-local-identifier"])
  |> count()
  |> sort(columns: ["_value"], desc: true)
  |> limit(n: 3)
'''

result_top = query_api.query_data_frame(query_top_birds, org=INFLUX_ORG)

if isinstance(result_top, list) and len(result_top) > 0:
    df_top = pd.concat(result_top, ignore_index=True)
else:
    df_top = result_top

top_bird_ids = df_top['individual-local-identifier'].tolist()

print(f"🦅 Top 3 oiseaux les plus trackés:")
for i, bird_id in enumerate(top_bird_ids, 1):
    count = df_top[df_top['individual-local-identifier'] == bird_id]['_value'].values[0]
    print(f"  {i}. {bird_id}: {count:,} points")
print()

# Créer une carte centrée sur l'Europe
m = folium.Map(location=[50, 15], zoom_start=4, tiles='OpenStreetMap')

colors = ['red', 'blue', 'green']

for idx, bird_id in enumerate(top_bird_ids):
    # Récupérer la trajectoire complète
    query_traj = f'''
    from(bucket: "{INFLUX_BUCKET}")
      |> range(start: 0)
      |> filter(fn: (r) => r["_measurement"] == "bird_migration")
      |> filter(fn: (r) => r["individual-local-identifier"] == "{bird_id}")
      |> filter(fn: (r) => r["_field"] == "location-lat" or r["_field"] == "location-long")
      |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
      |> sort(columns: ["_time"])
    '''
    
    result_traj = query_api.query_data_frame(query_traj, org=INFLUX_ORG)
    
    if isinstance(result_traj, list) and len(result_traj) > 0:
        df_traj = pd.concat(result_traj, ignore_index=True)
    else:
        df_traj = result_traj
    
    if len(df_traj) > 0:
        # Créer la liste de points (lat, lon)
        points = list(zip(df_traj['location-lat'], df_traj['location-long']))
        
        # Ajouter la trajectoire avec animation
        from folium.plugins import AntPath
        AntPath(
            points,
            color=colors[idx],
            weight=2.5,
            opacity=0.8,
            delay=800,
            popup=f"<b>Oiseau: {bird_id}</b><br>{len(points)} points GPS"
        ).add_to(m)
        
        # Ajouter marqueur au début et à la fin
        folium.Marker(
            points[0],
            popup=f"Départ: {bird_id}",
            icon=folium.Icon(color=colors[idx], icon='play')
        ).add_to(m)
        
        folium.Marker(
            points[-1],
            popup=f"Fin: {bird_id}",
            icon=folium.Icon(color=colors[idx], icon='stop')
        ).add_to(m)
        
        print(f"✓ Trajectoire {bird_id} ({colors[idx]}): {len(points)} points")

print()
print("🗺️ Carte interactive générée ci-dessous:")
display(m)

### 6.2 Time-Series: Latitude au Fil du Temps

**Objectif** : Visualiser le pattern de migration saisonnière d'un oiseau (latitude vs temps).

In [ ]:
# Récupérer la série temporelle de latitude pour l'oiseau le plus tracké
main_bird = top_bird_ids[0]

query_lat_time = f'''
from(bucket: "{INFLUX_BUCKET}")
  |> range(start: 0)
  |> filter(fn: (r) => r["_measurement"] == "bird_migration")
  |> filter(fn: (r) => r["individual-local-identifier"] == "{main_bird}")
  |> filter(fn: (r) => r["_field"] == "location-lat")
  |> sort(columns: ["_time"])
'''

result_lat = query_api.query_data_frame(query_lat_time, org=INFLUX_ORG)

if isinstance(result_lat, list) and len(result_lat) > 0:
    df_lat = pd.concat(result_lat, ignore_index=True)
else:
    df_lat = result_lat

if len(df_lat) > 0:
    df_lat['_time'] = pd.to_datetime(df_lat['_time'])
    
    # Créer le graphique time-series
    fig, ax = plt.subplots(figsize=(16, 6))
    
    ax.plot(df_lat['_time'], df_lat['_value'], linewidth=1.5, color='#2E86AB', alpha=0.8)
    
    # Ajouter des bandes de couleur pour les saisons
    import matplotlib.patches as mpatches
    from datetime import datetime
    
    # Définir les saisons (approximatif)
    years = df_lat['_time'].dt.year.unique()
    for year in years:
        # Été (juin-août) - jaune clair
        summer_start = pd.Timestamp(f'{year}-06-01')
        summer_end = pd.Timestamp(f'{year}-08-31')
        ax.axvspan(summer_start, summer_end, alpha=0.2, color='yellow', label='Été' if year == years[0] else '')
        
        # Hiver (décembre-février)
        winter_start = pd.Timestamp(f'{year}-12-01')
        winter_end = pd.Timestamp(f'{year+1}-02-28')
        ax.axvspan(winter_start, winter_end, alpha=0.2, color='lightblue', label='Hiver' if year == years[0] else '')
    
    ax.set_xlabel('Date', fontsize=12)
    ax.set_ylabel('Latitude (°)', fontsize=12)
    ax.set_title(f'Pattern de Migration: Oiseau {main_bird}\\nLatitude au fil du temps (2009-2015)', fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.legend(loc='upper right')
    
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    print(f"📊 Observation:")
    print(f"   Les pics vers le haut (latitude haute) = été en Scandinavie")
    print(f"   Les creux vers le bas (latitude basse) = hiver en Afrique/Méditerranée")
    print(f"   → Pattern de migration saisonnière clairement visible!")
else:
    print("⚠️ Aucune donnée trouvée")

### 6.3 Heatmap: Densité des Points de Tracking en Fonction du Temps

**Objectif** : Visualiser l'évolution temporelle des zones de présence des oiseaux (migration saisonnière).

In [ ]:
# Récupérer tous les points GPS (échantillonnés pour performance)
query_all_points = f'''
from(bucket: "{INFLUX_BUCKET}")
  |> range(start: 0)
  |> filter(fn: (r) => r["_measurement"] == "bird_migration")
  |> filter(fn: (r) => r["_field"] == "location-lat" or r["_field"] == "location-long")
  |> sample(n: 1000)
  |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
'''

result_all = query_api.query_data_frame(query_all_points, org=INFLUX_ORG)

if isinstance(result_all, list) and len(result_all) > 0:
    df_all = pd.concat(result_all, ignore_index=True)
else:
    df_all = result_all

if len(df_all) > 0:
    # Créer la heatmap
    from folium.plugins import HeatMap
    
    m_heat = folium.Map(location=[45, 15], zoom_start=4, tiles='CartoDB positron')
    
    # Préparer les données pour la heatmap
    heat_data = [[row['location-lat'], row['location-long']] for idx, row in df_all.iterrows()]
    
    HeatMap(
        heat_data,
        radius=8,
        blur=10,
        max_zoom=6,
        gradient={0.4: 'blue', 0.65: 'lime', 0.8: 'yellow', 1.0: 'red'}
    ).add_to(m_heat)
    
    print(f"🔥 Heatmap générée avec {len(heat_data):,} points")
    print()
    print("📍 Interprétation des zones chaudes (rouge/jaune):")
    print("   - Zones de haute densité = aires de repos/nidification")
    print("   - Zones froides (bleu) = corridors de migration rapide")
    print("   - Permet d'identifier les sites importants pour la conservation")
    print()
    
    display(m_heat)
else:
    print("⚠️ Aucune donnée trouvée")

### 6.4 Corrélation: Latitude vs Couverture Végétale

**Objectif** : Y a-t-il une corrélation entre la latitude (zones géographiques) et la végétation ?

In [ ]:
# Récupérer les données de végétation avec coordonnées
query_veg = f'''
from(bucket: "{INFLUX_BUCKET}")
  |> range(start: 0)
  |> filter(fn: (r) => r["_measurement"] == "bird_migration")
  |> filter(fn: (r) => r["_field"] == "vegetation_cover_high" or r["_field"] == "vegetation_cover_low" or r["_field"] == "location-lat")
  |> sample(n: 1000)
  |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
'''

result_veg = query_api.query_data_frame(query_veg, org=INFLUX_ORG)

if isinstance(result_veg, list) and len(result_veg) > 0:
    df_veg = pd.concat(result_veg, ignore_index=True)
else:
    df_veg = result_veg

if len(df_veg) > 0 and 'vegetation_cover_high' in df_veg.columns:
    # Créer les scatter plots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Haute végétation vs latitude
    ax1.scatter(df_veg['location-lat'], df_veg['vegetation_cover_high'], 
                alpha=0.3, s=2, color='#2E7D32')
    ax1.set_xlabel('Latitude (°)', fontsize=11)
    ax1.set_ylabel('Couverture Haute Végétation', fontsize=11)
    ax1.set_title('Latitude vs Haute Végétation (forêts, arbres)', fontsize=13, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    
    # Calculer et afficher la corrélation
    corr_high = df_veg['location-lat'].corr(df_veg['vegetation_cover_high'])
    ax1.text(0.05, 0.95, f'Corrélation: {corr_high:.3f}', 
             transform=ax1.transAxes, fontsize=10, verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # Basse végétation vs latitude
    if 'vegetation_cover_low' in df_veg.columns:
        ax2.scatter(df_veg['location-lat'], df_veg['vegetation_cover_low'], 
                    alpha=0.3, s=2, color='#F57C00')
        ax2.set_xlabel('Latitude (°)', fontsize=11)
        ax2.set_ylabel('Couverture Basse Végétation', fontsize=11)
        ax2.set_title('Latitude vs Basse Végétation (prairies, cultures)', fontsize=13, fontweight='bold')
        ax2.grid(True, alpha=0.3)
        
        corr_low = df_veg['location-lat'].corr(df_veg['vegetation_cover_low'])
        ax2.text(0.05, 0.95, f'Corrélation: {corr_low:.3f}', 
                 transform=ax2.transAxes, fontsize=10, verticalalignment='top',
                 bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    plt.show()
    
    print("📊 Interprétation:")
    print(f"   Corrélation haute végétation: {corr_high:.3f}")
    if 'vegetation_cover_low' in df_veg.columns:
        print(f"   Corrélation basse végétation: {corr_low:.3f}")
    print()
    
    if corr_high < 0:
        print("   ✓ Corrélation négative haute végétation:")
        print("     → Latitudes élevées (Arctique/subarctique) = moins de forêts")
        print("     → Cohérent avec la géographie (toundra au nord)")
    else:
        print("   ✓ Pas de corrélation forte ou positive")
    
    print()
    print("💡 Utilité pour la biologie:")
    print("   - Comprendre les habitats préférés")
    print("   - Identifier les zones critiques pour la conservation")
    print("   - Prévoir l'impact du changement climatique sur les routes migratoires")
else:
    print("⚠️ Données de végétation non disponibles")

## 🎉 Fin du TP : Récapitulatif

### Ce que nous avons appris :

✅ **Concepts Time-Series** :
- Différence entre bases relationnelles, documents, et time-series
- Pourquoi InfluxDB pour le tracking GPS et les métriques

✅ **Schema Design InfluxDB** :
- **Tags vs Fields** : la décision la plus critique
- Impact de la cardinalité sur les performances
- Anti-patterns à éviter (tags constants, haute cardinalité)

✅ **Langage Flux** :
- Requêtes basiques (`from`, `range`, `filter`)
- Agrégations (`count`, `mean`, `group`)
- Window functions pour downsampling
- Pivot pour restructurer les données

✅ **Analyse Time-Series** :
- Requêtes pour trouver les extremes (min/max)
- Détection de patterns saisonniers avec agrégation temporelle
- Analyse comparative (été vs hiver)

✅ **Optimisation & Gestion** :
- Retention policies pour la gestion du cycle de vie des données
- Compression et efficacité du stockage
- Impact du schema design sur les performances

✅ **Visualisations (Démonstration)** :
- Cartes interactives de migration
- Time-series plots
- Heatmaps de densité
- Analyse de corrélation

### InfluxDB vs MongoDB : Quand utiliser quoi ?

| Critère | InfluxDB | MongoDB |
|---------|----------|---------|
| **Use case** | Métriques, IoT, tracking temporel | Documents flexibles, applications web |
| **Structure** | Tags + Fields + Timestamp | JSON documents |
| **Queries temporelles** | ⭐⭐⭐⭐⭐ Ultra-rapides | ⭐⭐⭐ Correct avec indexes |
| **Compression** | ⭐⭐⭐⭐⭐ 10-20x | ⭐⭐⭐ Moyenne |
| **Flexibilité schéma** | ⭐⭐ Tags/Fields fixes | ⭐⭐⭐⭐⭐ Total freedom |
| **Relations complexes** | ⭐ Pas adapté | ⭐⭐⭐ Avec $lookup |

**Choix pour notre projet :**
- ✅ InfluxDB : données temporelles, tracking GPS, métriques environnementales
- ❌ MongoDB : aurait fonctionné mais moins efficace pour les requêtes temporelles

### Prochaines étapes suggérées :

1. **Continuous Queries** : Pré-calculer des agrégations (downsampling automatique)
2. **Tasks** : Automatiser le traitement de données
3. **Alerting** : Détecter des patterns anormaux (oiseau hors zone attendue)
4. **Grafana** : Dashboards temps-réel pour monitoring
5. **Sharding & Replication** : Scalabilité horizontale pour très gros volumes

🔗 **Ressources** :
- [Documentation InfluxDB](https://docs.influxdata.com/influxdb/v2.0/)
- [Flux Language Guide](https://docs.influxdata.com/flux/v0.x/)
- [InfluxDB University](https://university.influxdata.com/)